In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
import copy

import re
from matplotlib import cm
import matplotlib.patches as patches

import os
from Bio import SeqIO
from Bio import Entrez

import scipy.stats
import math

In [ ]:
sequence_df = pd.read_csv('preprocess_datasets/preprocessed_datasets/2019_guo_nci60_formatted_peptide_quants.tsv',
                          sep = '\t', index_col = 0)
sequence_df

In [ ]:
all_proteins = sequence_df['Protein'].values
all_unique_proteins = np.unique(sequence_df['Protein'].values)
print("All proteins: ", len(all_unique_proteins))
print("All proteins: ", all_unique_proteins)

all_peptide_sequences = sequence_df['Peptide']

print("All sequences: ", len(all_peptide_sequences))
print("All sequences: ", all_peptide_sequences[:10])


## Download all the protein sequences

In [ ]:
Entrez.email = "abdincer@uw.edu"  # Always tell NCBI who you are

#Record protein sequences
all_protein_sequences = []
all_protein_information = []

for protein_id in all_unique_proteins:
    
    filename = "protein_fasta_files/" + protein_id + ".fasta"
    if not os.path.isfile(filename):
        # Downloading...
        net_handle = Entrez.efetch(
            db="protein", id=protein_id, rettype="fasta", retmode="text"
        )
        out_handle = open(filename, "w")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
        print("Saved")

    print("Parsing...")
    record = SeqIO.read(filename, "fasta")
    print(record)
    all_protein_sequences.append(record.seq)
    all_protein_information.append(record)

## Match peptide sequences with proteins

In [ ]:
#Record the indices for each peptide
peptide_sequence_start_indices = np.zeros(len(all_peptide_sequences))
peptide_sequence_end_indices = np.zeros(len(all_peptide_sequences))

for p_index, protein in enumerate(all_unique_proteins):
    
    protein_sequence = all_protein_sequences[p_index]
    print("Protein sequence: ", protein)
    print("Protein sequence: ", protein_sequence)
    
    peptides = all_peptide_sequences.iloc[np.where(all_proteins == protein)[0]]
    print(peptides)
    for peptide_index, peptide in enumerate(peptides):
        
        print("Peptide sequence:", peptide)
        print("Peptide sequence:", peptide_index)
        try:
            index_order = protein_sequence.index(peptide)
            print("Index of peptide:", index_order)
        except:
            print("Not detected")
        
        #Record the results
        print([np.where(sequence_df.index == peptides.index[peptide_index])[0]])
        peptide_sequence_start_indices[np.where(sequence_df.index == peptides.index[peptide_index])[0]] = index_order
        peptide_sequence_end_indices[np.where(sequence_df.index == peptides.index[peptide_index])[0]] = index_order + len(peptide)
        
        

In [ ]:
sequence_df['Peptide start index'] = peptide_sequence_start_indices
sequence_df['Peptide end index'] = peptide_sequence_end_indices

#Record results
peptide_indices = sequence_df[['Peptide', 'Protein', 'gene symbol', 'R2', 'best mscore',
       'numNA', 'Charge 2', 'Charge 3', 'Charge 4', 'Charge 5', 'Charge 6', 
                          'Peptide start index', 'Peptide end index']]

peptide_indices.index = sequence_df.index
peptide_indices

## Read abundance matrices

In [ ]:
output_filename = 'trained_models/2019_guo_nci60/2019_guo_nci60_'

#Read Q, alpha, and c matrices
Q_train = pd.read_csv(output_filename + 'Q_train.tsv', sep = '\t', index_col = 0)
Q_test = pd.read_csv(output_filename + 'Q_test.tsv', sep = '\t', index_col = 0)

P_train = pd.read_csv(output_filename + 'P_train.tsv', sep = '\t', index_col = 0)
P_test = pd.read_csv(output_filename + 'P_test.tsv', sep = '\t', index_col = 0)

predicted_train_coeffs = pd.read_csv(output_filename + 'coefficients_train.tsv', sep = '\t', index_col = 0)
predicted_test_coeffs = pd.read_csv(output_filename + 'coefficients_test.tsv', sep = '\t', index_col = 0)
predicted_train_coeffs.index = Q_train.index
predicted_test_coeffs.index = Q_test.index



## Create plots

In [ ]:
def createJointPlots(protein_name):
    
    ############################################
    # Define plot shapes
    SMALL_SIZE = 60
    MEDIUM_SIZE = 60
    BIGGER_SIZE = 90

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex = True, figsize=(40, 30))
    fig.tight_layout()
    
    
    ############################################
    #We want to sort the peptides and then get the abundances
    
    sorted_peptide_indices = peptide_indices.loc[sequence_df[sequence_df['Protein'] == protein_name].index].sort_values(by = ['Peptide start index', 'Peptide end index']).index
   
    #Get sorted raw abundances
    index_values = peptide_indices.loc[sorted_peptide_indices]['Peptide start index'].values
    index_values_end = peptide_indices.loc[sorted_peptide_indices]['Peptide end index'].values
    raw_peptide_abundances = Q_test.loc[sequence_df[sequence_df['Protein'] == protein_name].index].loc[sorted_peptide_indices]

    mean_raw_abundance = np.mean(raw_peptide_abundances.values)
    print("Mean raw abundance: ", mean_raw_abundance)
    
    #Plot the mean abundance across all runs for each peptide
    raw_peptide_abundances_run_mean = np.mean(raw_peptide_abundances.values, axis = 1)
   

    #Obtain adjusted abundances using coefficients
    sorted_peptide_indices = peptide_indices.loc[sequence_df[sequence_df['Protein'] == protein_name].index].sort_values(by = ['Peptide start index', 'Peptide end index']).index
    raw_peptide_abundances = Q_test.loc[sequence_df[sequence_df['Protein'] == protein_name].index].loc[sorted_peptide_indices]

    coeffs = predicted_test_coeffs.loc[sequence_df[sequence_df['Protein'] == protein_name].index].loc[sorted_peptide_indices].values.ravel().reshape((-1, 1))
    adjusted_peptide_abundances = raw_peptide_abundances / np.abs(coeffs)
    
    mean_adjusted_abundance = np.mean(adjusted_peptide_abundances.values)
    print("Mean adjusted abundance: ", mean_adjusted_abundance)
    
    #Plot the mean abundance across all runs for each peptide
    adjusted_peptide_abundances_run_mean = np.mean(adjusted_peptide_abundances.values, axis = 1)
    
    
    ############################################
    #Center all the raw and observed abundances
    
    raw_peptide_abundances_run_mean = raw_peptide_abundances_run_mean - mean_raw_abundance
    adjusted_peptide_abundances_run_mean = adjusted_peptide_abundances_run_mean - mean_adjusted_abundance
    
    mean_raw_abundance = np.mean(raw_peptide_abundances_run_mean)
    mean_adjusted_abundance = np.mean(adjusted_peptide_abundances_run_mean)
    
    
    ############################################
    #Plot the raw peptide abundances
    
    #Plot abundances of all peptides
    for i in range(len(raw_peptide_abundances_run_mean)):
        peptide_charges = peptide_indices.loc[sorted_peptide_indices].iloc[i]
          
        ax1.vlines(x = (index_values[i] + index_values_end[i]) / 2, 
                       ymin = mean_raw_abundance, ymax = raw_peptide_abundances_run_mean[i],
                   linewidth = 5, color = '#eb4d4b')
        
        ax1.scatter((index_values[i] + index_values_end[i]) / 2, raw_peptide_abundances_run_mean[i], alpha = 0.5, 
                    s = 1500, color = '#eb4d4b', zorder = 10)
        
    protein_sequence = all_protein_sequences[np.where(all_unique_proteins == protein_name)[0][0]]
    
    #Plot the mean raw peptide abundance
    ax1.plot(np.arange(len(protein_sequence)), 
              mean_raw_abundance * np.ones(len(protein_sequence)), 
              lw = 5, color = 'Black', label = 'Mean observed abundance')

    
    ############################################
    #Plot the adjusted peptide abundances
    
    for i in range(len(adjusted_peptide_abundances_run_mean)):
        peptide_charges = peptide_indices.loc[sorted_peptide_indices].iloc[i]
        
        ax2.vlines((index_values[i] + index_values_end[i]) / 2, 
                   ymin = mean_adjusted_abundance, ymax = adjusted_peptide_abundances_run_mean[i],
               linewidth = 5, color = '#8854d0')
        ax2.scatter((index_values[i] + index_values_end[i]) / 2, 
                    adjusted_peptide_abundances_run_mean[i], alpha = 0.5, 
                s = 1500, color = '#8854d0', zorder = 10)
        
    ax2.plot(np.arange(len(protein_sequence)), 
              mean_adjusted_abundance * np.ones(len(protein_sequence)), 
              lw = 5, color = 'Black', label = 'Mean adjusted abundance')

    
    ############################################
    #Plot the alignment of peptides along the protein, colored by charge 
    
    current_height = 0
    for charge in range(2, 7):
        
        #Plot each peptide as a bar plot
        current_end_indices = [0]
        available_heights = [0]

        for i in range(len(raw_peptide_abundances_run_mean)):
            peptide_charges = peptide_indices.loc[sorted_peptide_indices].iloc[i]

            if peptide_charges['Charge 2'] == 1:
                color_value = '#22a6b3'
            elif peptide_charges['Charge 3'] == 1:
                color_value = '#f0932b'
            elif peptide_charges['Charge 4'] == 1:
                color_value = '#f9ca24'
            elif peptide_charges['Charge 5'] == 1:
                color_value = '#be2edd'
            elif peptide_charges['Charge 6'] == 1:
                color_value = '#badc58'

            if peptide_charges['Charge ' + str(charge)] == 1:
                start_of_peptide = index_values[i]
                end_of_peptide = index_values_end[i]

                #Compare the start site to all heights
                plotted_flag = False
                for height in available_heights:
                    if not plotted_flag and start_of_peptide > current_end_indices[height]:
        
                        rect = patches.Rectangle((index_values[i], current_height + (height/2)), 
                                      index_values_end[i] - index_values[i], 0.4,
                                      facecolor = color_value, linewidth=2, 
                                      alpha = 0.5, edgecolor='black')
                        current_end_index = end_of_peptide
                        current_end_indices[height] = current_end_index
                        plotted_flag = True
                        ax3.add_patch(rect)
                    
                height = available_heights[-1]
                if not plotted_flag:
                    available_heights.append((height + 1))
                    height = height + 1
                    rect = patches.Rectangle((index_values[i], current_height + (height/2)), 
                                      index_values_end[i] - index_values[i], 0.4,
                                      facecolor = color_value, linewidth=2, 
                                      alpha = 0.5, edgecolor='black')
                    ax3.add_patch(rect)
                    current_end_indices.append(end_of_peptide)

            
        #Increase height 
        current_height = current_height + (available_heights[-1]/2) + 0.5
        
        
    ############################################
    #Plot cleavage points

    protein_sequence = all_protein_sequences[np.where(all_unique_proteins == protein_name)[0][0]]
    for i in range(len(protein_sequence)-1):
        if protein_sequence[i:i+1] == 'K' or protein_sequence[i:i+1] == 'R':
            
            ax3.vlines(x = i+1, 
                      ymin = -0.1, 
                      ymax = 0.0,
                      linewidth = 2,
                      color = 'black')
    
    
    ############################################
    #Define grids, labels, axes
    
    ax1.grid('minor')
    ax2.grid('minor')
    
    max_value = max(np.nanmax(raw_peptide_abundances_run_mean), 
                    np.nanmax(adjusted_peptide_abundances_run_mean)) * 1.25
    
    min_value = min(np.nanmin(raw_peptide_abundances_run_mean), 
                    np.nanmin(adjusted_peptide_abundances_run_mean)) * 1.5
    
    ax1.set_ylim([min_value, max_value])
    ax2.set_ylim([min_value, max_value])
    
    
    ax3.set_xlabel('Peptide index of protein ' + protein_name)
    ax1.set_ylabel('Centered \n observed \n abundances')
    ax2.set_ylabel('Centered \n adjusted \n abundances')
    ax3.set_ylabel('Peptides \n along \n the protein')
    
    raw_error = np.mean(np.abs(raw_peptide_abundances_run_mean - mean_raw_abundance)**2)
    adj_error = np.mean(np.abs(adjusted_peptide_abundances_run_mean - mean_adjusted_abundance)**2)
    percent_improvement = ((raw_error - adj_error) / raw_error) * 100
    
    #Count no of peptides that becomes closer to the mean
    count_peptide = np.where(np.abs(adjusted_peptide_abundances_run_mean - mean_adjusted_abundance) < np.abs(raw_peptide_abundances_run_mean - mean_raw_abundance))[0]
    print("No of peptides with smaller error: ", len(count_peptide))
    print("Percent of peptides with smaller error: ", (len(count_peptide) / len(adjusted_peptide_abundances_run_mean)))
    
    print("Error for raw values: ", raw_error)
    print("Error for adjusted values: ", adj_error)
    print("Percent improvement: ", percent_improvement)
    
    plt.xlim([-5, len(protein_sequence) + 5]) 

    ax3.set_ylim([0, current_height])
    
    plt.show()

In [ ]:
#Select one of the proteins containing many peptides
from collections import Counter

#Select one of the proteins containing many peptides
protein_names = sequence_df.loc[Q_test.index].groupby('Protein').count().sort_values(by = 'Peptide')[::-1][:40]
protein_names = protein_names.index
protein_names

In [ ]:
#Plots for top 10 test proteins
for i in range(10):
    createJointPlots(protein_names[i])
    